In [ ]:
%pip install gdown
import gdown

# URL for the folder
url = "https://drive.google.com/drive/folders/1NMwGhJQLLvIgv5QPgV6QpWp8r3TJxCuB?usp=share_link"
gdown.download_folder(url, quiet=True)

In [ ]:
!pip install tqdm
!pip install dlib

In [ ]:
import os
import zipfile
from tqdm import tqdm
import numpy as np
import dlib
import cv2
from google.colab.patches import cv2_imshow
import shutil

def extract_hog_features(image, image_path):
  gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
  hog = cv2.HOGDescriptor(
        _winSize=(gray.shape[1] // 8 * 8, gray.shape[0] // 8 * 8),  # Make sure to use divisible sizes
        _blockSize=(16, 16),  # Block size (typically 16x16 pixels)
        _blockStride=(8, 8),  # Block stride (usually 8)
        _cellSize=(8, 8),  # Cell size (usually 8x8)
        _nbins=9  # Number of bins in histogram (usually 9)
    )
  features = hog.compute(gray)
  if type(features) == tuple:
    return _, False

  return features.flatten(), True  # Flatten to a 1D vector for each region

def main():
  folder_path = "/content/reshaped_images"
  for file_name in os.listdir(folder_path):

    # zipped files in the folder biometric_systems_dataset
    if file_name.endswith(".zip"):

      # full path to the zip file
      file_path = os.path.join(folder_path, file_name)

      # create folder in the same path with the name of the zip file
      extract_folder = os.path.join(folder_path, file_name.replace(".zip", ""))
      os.makedirs(extract_folder, exist_ok=True)

      # open and extract the file
      with zipfile.ZipFile(file_path, mode="r") as zip_ref:
        zip_ref.extractall(extract_folder)

  feature_folders = ["eyes", "nose", "mouth"]
  bad_images = []
  for feature in feature_folders:
    feature_folder_path = os.path.join(folder_path, feature)

    #remove the extracted zip files
    if feature_folder_path.endswith(".zip"):
      os.remove(feature_folder_path)

    # create embeddings directories
    embeddings_folder = os.path.join(f"embeddings_{feature}")
    os.makedirs(embeddings_folder, exist_ok=True)
    for file_name in tqdm(os.listdir(feature_folder_path), desc="Processing Images"):
      if file_name.endswith(".jpg"):
        image_path = os.path.join(feature_folder_path, file_name)
        image = cv2.imread(image_path)
        hog_features, boolean = extract_hog_features(image, image_path)
        if boolean:
          np.save(os.path.join(embeddings_folder, file_name.replace(".jpg", ".npy")), hog_features)
        else:
          bad_images.append(image_path)
    shutil.make_archive(embeddings_folder, "zip", embeddings_folder)

  for image in bad_images:
    os.remove(image)

if __name__ == "__main__":
  main()